# Media-Pipe Preprocessing

## Convert Multiple Raw Videos into Mediapipe Stick-Figures

In [1]:

import cv2
import mediapipe as mp
import numpy as np
import time
import os
import glob

parent_directory='data'
video_dir='mero_dherei_sathi_haru_chhan/'


video_path=os.path.join(parent_directory,video_dir)


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands


prev_frame_time=0
new_frame_time=0

# For webcam input:
# cap = cv2.VideoCapture(0)


for video in glob.glob(video_path+'*'):

  cap=cv2.VideoCapture(video)

  fourcc = cv2.VideoWriter_fourcc('X','V','I','D')

  save_video_path=os.path.join('processed_data/'+video_dir+video.removeprefix(video_path))
  videoWriter = cv2.VideoWriter(save_video_path, fourcc, 30.0, (224,224))


  with mp_pose.Pose(
      min_detection_confidence=0.5,
      min_tracking_confidence=0.5) as pose:
    with mp_hands.Hands(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
      while cap.isOpened():
        success, image = cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          break
          # If loading a video, use 'break' instead of 'continue'.
          # continue

        image=cv2.resize(image,(224,224),interpolation=cv2.INTER_AREA)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        results_p=pose.process(image)


        img=np.zeros_like(image,dtype=np.uint8)

        mp_drawing.draw_landmarks(
        img,
        results_p.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),hands_or_pose='Pose')  

        if results.multi_hand_landmarks:
          for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                img,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style(),
                hands_or_pose='Hands')


        new_frame_time=time.time()
        fps = str(int(1/(new_frame_time-prev_frame_time)))
        prev_frame_time = new_frame_time

        # Flip the image horizontally for a selfie-view display.
        image=cv2.flip(image,1)
        img=cv2.flip(img,1)

        # save the mediapipe hand and pose video
        videoWriter.write(img)

        cv2.putText(image, fps, (7, 70), cv2.FONT_HERSHEY_SIMPLEX , 3, (100, 255, 0), 3, cv2.LINE_AA)
        cv2.imshow('MediaPipe Hand and Pose', img)      
        cv2.imshow('Original',cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        if cv2.waitKey(5) & 0xFF == 27:
          break
  cap.release()
  videoWriter.release()
cv2.destroyAllWindows()

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
I0000 00:00:1723633160.329710    6930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723633160.333018    6990 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1723633160.342759    6930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723633160.345027    7001 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
W0000 00:00:1723633160.369530    6991 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723633160.402133    6998 infe

Ignoring empty camera frame.


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
I0000 00:00:1723633171.866237    6930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723633171.867286    7031 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
I0000 00:00:1723633171.875009    6930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723633171.875914    7042 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
W0000 00:00:1723633171.892148    7036 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723633171.922045    7034 inference_feedback_manager.cc:114] Feedback manager requires

Ignoring empty camera frame.


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
I0000 00:00:1723633183.453017    6930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723633183.454342    7083 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
I0000 00:00:1723633183.464414    6930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723633183.465381    7094 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
W0000 00:00:1723633183.479984    7088 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723633183.508215    7090 inference_feedback_manager.cc:114] Feedback manager requires

...

Ignoring empty camera frame.


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
I0000 00:00:1723633229.780174    6930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723633229.781253    7290 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
I0000 00:00:1723633229.789971    6930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723633229.791038    7301 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
W0000 00:00:1723633229.806368    7294 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723633229.833975    7294 inference_feedback_manager.cc:114] Feedback manager requires

Ignoring empty camera frame.


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
I0000 00:00:1723633242.992682    6930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723633242.994300    7342 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
I0000 00:00:1723633243.003170    6930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723633243.004123    7353 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
W0000 00:00:1723633243.019651    7349 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723633243.046071    7346 inference_feedback_manager.cc:114] Feedback manager requires

Ignoring empty camera frame.


## Real Time Stick-Figure Generation From Webcam Input

In [6]:

import cv2
import mediapipe as mp
import numpy as np
import time
import os


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands


prev_frame_time = 0
new_frame_time = 0


# For webcam input:
cap = cv2.VideoCapture('m_dherei_sathi_hc.mp4')

fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')


videoWriter = cv2.VideoWriter('slomo1.avi', fourcc, 15.0, (224,224))

with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    with mp_hands.Hands(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                break
                # If loading a video, use 'break' instead of 'continue'.
                # continue

            image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            results_p = pose.process(image)

            img = np.zeros_like(image, dtype=np.uint8)

            mp_drawing.draw_landmarks(
                img,
                results_p.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(), hands_or_pose='Pose')

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        img,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style(),
                        hands_or_pose='Hands')

            new_frame_time = time.time()
            fps = str(int(1/(new_frame_time-prev_frame_time)))
            prev_frame_time = new_frame_time

            # Flip the image horizontally for a selfie-view display.
            image = cv2.flip(image, 1)
            img = cv2.flip(img, 1)

            # save the mediapipe hand and pose video
            # videoWriter.write(image)

            cv2.putText(image, fps, (7, 70), cv2.FONT_HERSHEY_SIMPLEX,
                        3, (100, 255, 0), 3, cv2.LINE_AA)
            cv2.imshow('MediaPipe Hand and Pose', img)
            cv2.imshow('Original', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
            if cv2.waitKey(5) & 0xFF == 27:
                break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

I0000 00:00:1723805563.886720    5920 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723805563.887895    8000 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
I0000 00:00:1723805563.899278    5920 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723805563.900711    8011 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (ICL GT1)
W0000 00:00:1723805563.917761    8002 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723805563.940920    8008 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723805563.992520    7991 inference_feedback_manager.cc:114] Feedback 

## making the dataset

In [5]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path

In [ ]:
म घर मा धेरै काम गर्छु । मेरो घर भक्तपुर मा छ । म संग धेरै पैसा छैन । मेरो धेरै साथी हरु छन् । मेरो साथी हरु भक्तपुर मा छन् ।

In [2]:
ds={'ne':['म घर मा धेरै काम गर्छु ।','मेरो घर भक्तपुर मा छ ।','म संग धेरै पैसा छैन ।','मेरो धेरै साथी हरु छन् ।','मेरो साथी हरु भक्तपुर मा छन् ।']}
ds

{'ne': ['म घर मा धेरै काम गर्छु ।',
  'मेरो घर भक्तपुर मा छ ।',
  'म संग धेरै पैसा छैन ।',
  'मेरो धेरै साथी हरु छन् ।',
  'मेरो साथी हरु भक्तपुर मा छन् ।']}

In [8]:
def get_all_sentences(ds,lang):
    for item in ds[lang]:
        yield item


def get_or_build_tokenizer(filename, ds, lang):
    tokenizer_path = Path(filename.format(lang))
    if not Path.exists(tokenizer_path):
        # Most code taken from: https://huggingface.co/docs/tokenizers/quicktour
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(
            special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=1)
        tokenizer.train_from_iterator(
            get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))

    return tokenizer

In [15]:
# making the tokenizer

tok=get_or_build_tokenizer('tokenizer_sign_lang_{0}.json',ds,'ne')

{'छैन': 17,
 '[SOS]': 2,
 '।': 4,
 'छन्': 9,
 'म': 11,
 'संग': 19,
 'साथी': 12,
 '[UNK]': 0,
 'मेरो': 7,
 'हरु': 13,
 '[PAD]': 1,
 '[EOS]': 3,
 'काम': 14,
 'गर्छु': 15,
 'मा': 6,
 'छ': 16,
 'धेरै': 5,
 'भक्तपुर': 10,
 'पैसा': 18,
 'घर': 8}

## random clipping

In [ ]:
!pip install -q pytorchvideo transformers evaluate

In [ ]:
import pytorchvideo

pytorchvideo.data.make_clip_sampler("random", clip_duration)